### YOLO-Only Testing

In [2]:
# %pip install -r requirements.txt

In [1]:
from src.scene_cutting import get_scene_list
from src.frame_sampling import sample_frames
from src.yolo_inference import run_yolo_on_frames
from src.debug_utils import see_first_scene, see_scenes_cuts
from src.perf_utils import measure_performance

test_video = r"Videos\SpongeBob SquarePants - Writing Essay - Some of These - Meme Source.mp4"

c:\Users\manah\anaconda3\envs\Kairos\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#functions 
get_scene_list = measure_performance(get_scene_list)
sample_frames = measure_performance(sample_frames)
run_yolo_on_frames = measure_performance(run_yolo_on_frames)

In [3]:
# Total pipeline start
import time, psutil, os
process = psutil.Process(os.getpid())
mem_start = process.memory_info().rss / (1024**2)
t_start = time.time()

In [4]:
# Scene detection
scenes = get_scene_list(test_video)
see_scenes_cuts(scenes)

VideoManager is deprecated and will be removed.
`base_timecode` argument is deprecated and has no effect.


[PERF] get_scene_list -> Time: 4.58s, Memory: 8.27 MB (delta)
Detected scenes:
Scene 0: start=0 end=118 frames=0
Scene 1: start=118 end=187 frames=0
Scene 2: start=187 end=248 frames=0
Scene 3: start=248 end=300 frames=0
Scene 4: start=300 end=577 frames=0
Scene 5: start=577 end=629 frames=0
Scene 6: start=629 end=785 frames=0
Scene 7: start=785 end=827 frames=0
Scene 8: start=827 end=869 frames=0
Scene 9: start=869 end=870 frames=0


In [5]:
# Frame sampling
scenes_with_frames = sample_frames(
    input_video_path=test_video,
    scenes=scenes,
    num_frames=2,
    output_dir="./output/frames",
)
scenes_with_frames[:1]

[PERF] sample_frames -> Time: 2.23s, Memory: 1.80 MB (delta)


[{'start': 0,
  'end': 118,
  'frames': ['./output/frames\\scene0_frame0.jpg',
   './output/frames\\scene0_frame1.jpg',
   './output/frames\\scene0_frame2.jpg']}]

In [6]:
# YOLO inference
yolo_results = run_yolo_on_frames(
    scenes=scenes_with_frames,
    model_size="yolov8s",
    conf=0.25,
    iou=0.45,
)

[PERF] run_yolo_on_frames -> Time: 11.15s, Memory: 165.59 MB (delta)


In [7]:
# Debug visualization
see_first_scene(yolo_results)

First scene frame info: {'path': './output/frames\\scene0_frame0.jpg', 'detections': [{'label': 'apple', 'confidence': 0.49817800521850586, 'bbox': [161.191162109375, 532.7568359375, 356.44830322265625, 704.0435791015625]}, {'label': 'suitcase', 'confidence': 0.35428425669670105, 'bbox': [563.3948974609375, 0.97784423828125, 841.5506591796875, 532.7828369140625]}, {'label': 'dining table', 'confidence': 0.3053893446922302, 'bbox': [162.63235473632812, 528.18994140625, 1103.457763671875, 709.580810546875]}]}


In [8]:
# Total pipeline performance
t_end = time.time()
mem_end = process.memory_info().rss / (1024**2)
print(f"[PIPELINE PERF] Total time: {t_end - t_start:.2f}s, "
      f"Total memory delta: {mem_end - mem_start:.2f} MB")

[PIPELINE PERF] Total time: 18.96s, Total memory delta: 175.75 MB
